<a href="https://colab.research.google.com/github/qqfish/tensorboard/blob/fishx-profiler-tutorial/tensorboard/docs/r2/tensorboard_profiling_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorBoard Profile: Profiling basic training metrics in Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Currently N/A
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorboard/blob/master/docs/r2/tensorboard_profiling_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorboard/blob/master/docs/r2/tensorboard_profiling_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


## Overview
Performance is critical for machine learning. TensorFlow has a built-in profiler allows you to records runtime of each ops with very little effort. Then you can visualize the profile result in TensorBoard's **Profile Plugin**.

This tutorial presents very basic examples to help you learn how to use profiler APIs when developing your Keras model. You will learn how to use the Keras TensorBoard callback to visualize profile result.

## Setup

Select **GPU** in the Accelerator drop-down in Notebook Settings.

![Notebook Settings](https://screenshot.googleplex.com/D0OytHGqonX.png)

Install latest TensorFlow.

In [0]:
# Ensure latest TensorFlow is installed.
!pip install -q tf-nightly-gpu
# Load the TensorBoard notebook extension.
%load_ext tensorboard


    100% |████████████████████████████████| 374.2MB 45kB/s 
    100% |████████████████████████████████| 3.1MB 7.2MB/s 
    100% |████████████████████████████████| 481kB 4.8MB/s 
    100% |████████████████████████████████| 61kB 27.2MB/s 
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

import functools
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.python.keras import backend
from tensorflow.python.keras import layers

import numpy as np

print("TensorFlow version: ", tf.__version__)
tf.enable_v2_behavior()

TensorFlow version:  1.14.1-dev20190424


Confirm TensorFlow can see the GPU.

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Run a simple model with TensorBoard callback

You're now going to use Keras to build a simple model for classifying [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) images using ResNet56 (Reference: [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)).

Following RestNet model code is copied from [TensorFlow models garden](https://github.com/tensorflow/models/blob/master/official/resnet/keras/resnet_cifar_model.py).



In [0]:
BATCH_NORM_DECAY = 0.997
BATCH_NORM_EPSILON = 1e-5
L2_WEIGHT_DECAY = 2e-4


def identity_building_block(input_tensor,
                            kernel_size,
                            filters,
                            stage,
                            block,
                            training=None):
  """The identity block is the block that has no conv layer at shortcut.

  Arguments:
    input_tensor: input tensor
    kernel_size: default 3, the kernel size of
        middle conv layer at main path
    filters: list of integers, the filters of 3 conv layer at main path
    stage: integer, current stage label, used for generating layer names
    block: current block label, used for generating layer names
    training: Only used if training keras model with Estimator.  In other
      scenarios it is handled automatically.

  Returns:
    Output tensor for the block.
  """
  filters1, filters2 = filters
  if tf.keras.backend.image_data_format() == 'channels_last':
    bn_axis = 3
  else:
    bn_axis = 1
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  x = tf.keras.layers.Conv2D(filters1, kernel_size,
                             padding='same',
                             kernel_initializer='he_normal',
                             kernel_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             bias_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             name=conv_name_base + '2a')(input_tensor)
  x = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                         name=bn_name_base + '2a',
                                         momentum=BATCH_NORM_DECAY,
                                         epsilon=BATCH_NORM_EPSILON)(
                                             x, training=training)
  x = tf.keras.layers.Activation('relu')(x)

  x = tf.keras.layers.Conv2D(filters2, kernel_size,
                             padding='same',
                             kernel_initializer='he_normal',
                             kernel_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             bias_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             name=conv_name_base + '2b')(x)
  x = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                         name=bn_name_base + '2b',
                                         momentum=BATCH_NORM_DECAY,
                                         epsilon=BATCH_NORM_EPSILON)(
                                             x, training=training)

  x = tf.keras.layers.add([x, input_tensor])
  x = tf.keras.layers.Activation('relu')(x)
  return x


def conv_building_block(input_tensor,
                        kernel_size,
                        filters,
                        stage,
                        block,
                        strides=(2, 2),
                        training=None):
  """A block that has a conv layer at shortcut.

  Arguments:
    input_tensor: input tensor
    kernel_size: default 3, the kernel size of
        middle conv layer at main path
    filters: list of integers, the filters of 3 conv layer at main path
    stage: integer, current stage label, used for generating layer names
    block: current block label, used for generating layer names
    strides: Strides for the first conv layer in the block.
    training: Only used if training keras model with Estimator.  In other
      scenarios it is handled automatically.

  Returns:
    Output tensor for the block.

  Note that from stage 3,
  the first conv layer at main path is with strides=(2, 2)
  And the shortcut should have strides=(2, 2) as well
  """
  filters1, filters2 = filters
  if tf.keras.backend.image_data_format() == 'channels_last':
    bn_axis = 3
  else:
    bn_axis = 1
  conv_name_base = 'res' + str(stage) + block + '_branch'
  bn_name_base = 'bn' + str(stage) + block + '_branch'

  x = tf.keras.layers.Conv2D(filters1, kernel_size, strides=strides,
                             padding='same',
                             kernel_initializer='he_normal',
                             kernel_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             bias_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             name=conv_name_base + '2a')(input_tensor)
  x = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                         name=bn_name_base + '2a',
                                         momentum=BATCH_NORM_DECAY,
                                         epsilon=BATCH_NORM_EPSILON)(
                                             x, training=training)
  x = tf.keras.layers.Activation('relu')(x)

  x = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same',
                             kernel_initializer='he_normal',
                             kernel_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             bias_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             name=conv_name_base + '2b')(x)
  x = tf.keras.layers.BatchNormalization(axis=bn_axis,
                                         name=bn_name_base + '2b',
                                         momentum=BATCH_NORM_DECAY,
                                         epsilon=BATCH_NORM_EPSILON)(
                                             x, training=training)

  shortcut = tf.keras.layers.Conv2D(filters2, (1, 1), strides=strides,
                                    kernel_initializer='he_normal',
                                    kernel_regularizer=
                                    tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                    bias_regularizer=
                                    tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                                    name=conv_name_base + '1')(input_tensor)
  shortcut = tf.keras.layers.BatchNormalization(
      axis=bn_axis, name=bn_name_base + '1',
      momentum=BATCH_NORM_DECAY, epsilon=BATCH_NORM_EPSILON)(
          shortcut, training=training)

  x = tf.keras.layers.add([x, shortcut])
  x = tf.keras.layers.Activation('relu')(x)
  return x


def resnet_block(input_tensor,
                 size,
                 kernel_size,
                 filters,
                 stage,
                 conv_strides=(2, 2),
                 training=None):
  """A block which applies conv followed by multiple identity blocks.

  Arguments:
    input_tensor: input tensor
    size: integer, number of constituent conv/identity building blocks.
    A conv block is applied once, followed by (size - 1) identity blocks.
    kernel_size: default 3, the kernel size of
        middle conv layer at main path
    filters: list of integers, the filters of 3 conv layer at main path
    stage: integer, current stage label, used for generating layer names
    conv_strides: Strides for the first conv layer in the block.
    training: Only used if training keras model with Estimator.  In other
      scenarios it is handled automatically.

  Returns:
    Output tensor after applying conv and identity blocks.
  """

  x = conv_building_block(input_tensor, kernel_size, filters, stage=stage,
                          strides=conv_strides, block='block_0',
                          training=training)
  for i in range(size - 1):
    x = identity_building_block(x, kernel_size, filters, stage=stage,
                                block='block_%d' % (i + 1), training=training)
  return x

def resnet(num_blocks, classes=10, training=None):
  """Instantiates the ResNet architecture.

  Arguments:
    num_blocks: integer, the number of conv/identity blocks in each block.
      The ResNet contains 3 blocks with each block containing one conv block
      followed by (layers_per_block - 1) number of idenity blocks. Each
      conv/idenity block has 2 convolutional layers. With the input
      convolutional layer and the pooling layer towards the end, this brings
      the total size of the network to (6*num_blocks + 2)
    classes: optional number of classes to classify images into
    training: Only used if training keras model with Estimator.  In other
    scenarios it is handled automatically.

  Returns:
    A Keras model instance.
  """

  input_shape = (32, 32, 3)
  img_input = layers.Input(shape=input_shape)

  if backend.image_data_format() == 'channels_first':
    x = layers.Lambda(lambda x: backend.permute_dimensions(x, (0, 3, 1, 2)),
                      name='transpose')(img_input)
    bn_axis = 1
  else:  # channel_last
    x = img_input
    bn_axis = 3

  x = tf.keras.layers.ZeroPadding2D(padding=(1, 1), name='conv1_pad')(x)
  x = tf.keras.layers.Conv2D(16, (3, 3),
                             strides=(1, 1),
                             padding='valid',
                             kernel_initializer='he_normal',
                             kernel_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             bias_regularizer=
                             tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                             name='conv1')(x)
  x = tf.keras.layers.BatchNormalization(axis=bn_axis, name='bn_conv1',
                                         momentum=BATCH_NORM_DECAY,
                                         epsilon=BATCH_NORM_EPSILON)(
                                             x, training=training)
  x = tf.keras.layers.Activation('relu')(x)

  x = resnet_block(x, size=num_blocks, kernel_size=3, filters=[16, 16],
                   stage=2, conv_strides=(1, 1), training=training)

  x = resnet_block(x, size=num_blocks, kernel_size=3, filters=[32, 32],
                   stage=3, conv_strides=(2, 2), training=training)

  x = resnet_block(x, size=num_blocks, kernel_size=3, filters=[64, 64],
                   stage=4, conv_strides=(2, 2), training=training)

  x = tf.keras.layers.GlobalAveragePooling2D(name='avg_pool')(x)
  x = tf.keras.layers.Dense(classes, activation='softmax',
                            kernel_initializer='he_normal',
                            kernel_regularizer=
                            tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                            bias_regularizer=
                            tf.keras.regularizers.l2(L2_WEIGHT_DECAY),
                            name='fc10')(x)

  inputs = img_input
  # Create model.
  model = tf.keras.models.Model(inputs, x, name='resnet56')

  return model


resnet20 = functools.partial(resnet, num_blocks=3)
resnet32 = functools.partial(resnet, num_blocks=5)
resnet56 = functools.partial(resnet, num_blocks=9)
resnet10 = functools.partial(resnet, num_blocks=110)

Download CIFAR-10 data from [TensorFlow Datasets](https://www.tensorflow.org/datasets).

In [0]:
cifar_builder = tfds.builder('cifar10')
cifar_builder.download_and_prepare()


W0424 18:29:27.735397 140204269844352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:325: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/1.0.2. Subsequent calls will reuse this data.


Build data input pipeline and compile ResNet56 model.

In [0]:
HEIGHT = 32
WIDTH = 32
NUM_CHANNELS = 3
NUM_CLASSES = 10
BATCH_SIZE = 128

def preprocess_data(record):
  image = record['image']
  label = record['label']
  
  # Resize the image to add four extra pixels on each side.
  image = tf.image.resize_image_with_crop_or_pad(
      image, HEIGHT + 8, WIDTH + 8)

  # Randomly crop a [HEIGHT, WIDTH] section of the image.
  image = tf.image.random_crop(image, [HEIGHT, WIDTH, NUM_CHANNELS])

  # Randomly flip the image horizontally.
  image = tf.image.random_flip_left_right(image)

  # Subtract off the mean and divide by the variance of the pixels.
  image = tf.image.per_image_standardization(image)
  
  label = tf.compat.v1.sparse_to_dense(label, (NUM_CLASSES,), 1)
  return image, label

train_data = cifar_builder.as_dataset(split=tfds.Split.TRAIN)
train_data = train_data.repeat()
train_data = train_data.map(
    lambda value: preprocess_data(value))
train_data = train_data.shuffle(1024)

train_data = train_data.batch(BATCH_SIZE)

model = resnet56(classes=NUM_CLASSES)

model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/readers.py:81: UserWarning: Creating resources inside a function passed to Dataset.flat_map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  return filenames.flat_map(read_one_file)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:1780: UserWarning: Creating resources inside a function passed to Dataset.interleave() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  map_func, cycle_length, block_length, num_parallel_calls))
W0424 18:29:50.257898 140204269844352 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/image_ops_impl.py:1444: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0424 18:29:50.261064 140204269

When creating TensorBoard callback, you can specify the batch num you want to profile. By default, TensorFlow will profile the second batch, because many one time graph optimizations run on the first batch. You can modify it by setting **profile_batch**. You can also turn off profiling by setting it to 0.

This time, you will profile on the third batch.

In [0]:
log_dir="logs/profile/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = 3)

W0424 18:30:01.045316 140204269844352 callbacks_v1.py:135] Weight and gradient histograms not supported for eagerexecution, setting `histogram_freq` to `0`.


Start training by calling [Model.fit()](https://https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#fit). 

In [0]:
model.fit(train_data,
          steps_per_epoch=20,
          epochs=5, 
          callbacks=[tensorboard_callback])

Epoch 1/5
 1/20 [>.............................] - ETA: 11:51 - loss: 5.4309 - categorical_accuracy: 0.0781

W0424 18:30:44.176247 140204269844352 callbacks.py:238] Method (on_train_batch_end) is slow compared to the batch update (0.321069). Check your callbacks.


20/20 [==============================] - 43s 2s/step - loss: 4.4876 - categorical_accuracy: 0.1000
Epoch 2/5
20/20 [==============================] - 4s 220ms/step - loss: 3.8578 - categorical_accuracy: 0.1191
Epoch 3/5
20/20 [==============================] - 5s 238ms/step - loss: 3.5981 - categorical_accuracy: 0.1086
Epoch 4/5
20/20 [==============================] - 5s 238ms/step - loss: 3.5089 - categorical_accuracy: 0.1168
Epoch 5/5
20/20 [==============================] - 4s 224ms/step - loss: 3.4146 - categorical_accuracy: 0.1223


## Visualizing profile result using TensorBoard

Unfortunately, due to a [issue#1913](https://github.com/tensorflow/tensorboard/issues/1913), you cannot use TensorBoard in Colab to visualize profile result. You are going to download the logdir and start TensorBoard on your local machine.

Compress logdir:


In [0]:
!tar -zcvf logs.tar.gz logs/profile/

logs/profile/
logs/profile/20190424-183001/
logs/profile/20190424-183001/events.out.tfevents.1556130605.ea01b31b088f.v2
logs/profile/20190424-183001/plugins/
logs/profile/20190424-183001/plugins/profile/
logs/profile/20190424-183001/plugins/profile/2019-04-24_18-30-44/
logs/profile/20190424-183001/plugins/profile/2019-04-24_18-30-44/local.trace
logs/profile/20190424-183001/events.out.tfevents.1556130644.ea01b31b088f.profile-empty


Download **logdir.tar.gz** by right-clicking it in **Files** tab.

![Download](https://screenshot.googleplex.com/U12ehqdiqQ0.png)

Please make sure you have the latest [TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard#setup) install at you local machine as well. Execute following commands on your local machine:

```
> cd download/directory
> tar -zxvf logs.tar.gz
> tensorboard --logdir=logs/ --port=6006

```




Open a new tab in your Chrome browser and navigate to [localhost:6006](http://localhost:6006) and then click **Profile** tab. You may see the profile result like this:

![Trace View](https://screenshot.googleplex.com/ynOHdaHPJeg.png)




## Trace Viewer
The page you see on TensorBoard, we called it Trace Viewer. This is the major tool experienced TensorFlow developers used to debug performance issues inside Google. It displays a timeline of different events that happened on the CPU and the accelerator during the collection period.

The Trace Viewer shows multiple **event groups** on the vertical axis. Each event group appears in a separate horizontal **track**, filled with trace events. The **track** is basically an event timeline for events on a thread/process. Events are the colored, rectangular blocks on the timeline tracks. Time moves from left to right.

You can navigate through the result using **w** (zoom in), **s** (zoom out), **a** (scroll left), **d** (scroll right).

A single rectangle represents a **trace event**: when it began, and when it ended. To study an individual rectangle, you can click on it. This will display information about the rectangle, such as its Start time and Duration.

In addition to clicking, you can drag the mouse in a rectangle over a group of trace events. This will give you a list of events that intersect that rectangle and summarize them for you. The **m** key can be used to mark the selection and determine its total duration.

![List of Events](https://screenshot.googleplex.com/6qDtyNSc0ks.png)

The trace events are collected from three sources:


*   **CPU**: CPU events are under event group named **/host:CPU**. Each track represents a thread on CPU. E.g. input pipeline events, GPU op scheduling events, CPU ops execution events, etc.
*   **GPU**: GPU events are under event groups prefixed by **/device:GPU:***.  Except **stream:all**, each event group represents one stream on GPU. **stream::all** aggregates all events on one GPU. E.g. Memory copy events, Kernel execution events, etc.
*   **TensorFlow Runtime**: Runtime events are under event groups prefixed by **/job:***. Runtime events represent the TensorFlow ops invoked by python program. E.g. tf.function execution events, etc.

## Debug Performance
Now, you're going to use the Trace Viewer to improve your model's performance. 

Let's go back to the profile result you have just captured.
![GPU kernel](https://screenshot.googleplex.com/G4obPVhVNwP.png)

GPU events show that GPU has nothing to do at all in the first harf of the step.

![CPU events](https://screenshot.googleplex.com/OByctgNgve4.png)

CPU events show that CPU is occupied by data input pipeline in the beginning of this step.

![Runtime](https://screenshot.googleplex.com/whPYibeWpRP.png)

In TensorFlow runtime, there is a big block named **Iterator::GetNextSync**, which is a blocking call to get the next batch from data input pipeline. And it blocks the training step. So if you could prepare the input data for step **s** in **s-1** step, you can probably train this model faster.

You can achieve it by using [tf.data.prefetch].(https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch)





In [0]:
train_data = cifar_builder.as_dataset(split=tfds.Split.TRAIN)
train_data = train_data.repeat()
train_data = train_data.map(
    lambda value: preprocess_data(value))
train_data = train_data.shuffle(1024)
train_data = train_data.batch(BATCH_SIZE)

# It will prefetch the data in (s-1) step
train_data = train_data.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/readers.py:81: UserWarning: Creating resources inside a function passed to Dataset.flat_map() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  return filenames.flat_map(read_one_file)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:1780: UserWarning: Creating resources inside a function passed to Dataset.interleave() is not supported. Create each resource outside the function, and capture it inside the function to use it.
  map_func, cycle_length, block_length, num_parallel_calls))


Re-run the model.

In [0]:
log_dir="logs/profile/" + datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch = 3)

model.fit(train_data,
          steps_per_epoch=20,
          epochs=5, 
          callbacks=[tensorboard_callback])

W0424 18:38:58.577626 140204269844352 callbacks_v1.py:135] Weight and gradient histograms not supported for eagerexecution, setting `histogram_freq` to `0`.


Epoch 1/5
20/20 [==============================] - 6s 275ms/step - loss: 3.1681 - categorical_accuracy: 0.1496
Epoch 2/5
20/20 [==============================] - 4s 199ms/step - loss: 3.1580 - categorical_accuracy: 0.1512
Epoch 3/5
20/20 [==============================] - 4s 190ms/step - loss: 3.1154 - categorical_accuracy: 0.1477
Epoch 4/5
20/20 [==============================] - 4s 196ms/step - loss: 3.1181 - categorical_accuracy: 0.1523
Epoch 5/5
20/20 [==============================] - 4s 195ms/step - loss: 3.0973 - categorical_accuracy: 0.1590


Woohoo! You have just improvd training performance from **~230ms/step** to **~195ms/step**.  

In [0]:
!tar -zcvf logs.tar.gz logs/profile/

logs/profile/
logs/profile/20190424-183001/
logs/profile/20190424-183001/events.out.tfevents.1556130605.ea01b31b088f.v2
logs/profile/20190424-183001/plugins/
logs/profile/20190424-183001/plugins/profile/
logs/profile/20190424-183001/plugins/profile/2019-04-24_18-30-44/
logs/profile/20190424-183001/plugins/profile/2019-04-24_18-30-44/local.trace
logs/profile/20190424-183001/events.out.tfevents.1556130644.ea01b31b088f.profile-empty
logs/profile/20190424-183727/
logs/profile/20190424-183727/events.out.tfevents.1556131050.ea01b31b088f.profile-empty
logs/profile/20190424-183727/events.out.tfevents.1556131047.ea01b31b088f.v2
logs/profile/20190424-183727/plugins/
logs/profile/20190424-183727/plugins/profile/
logs/profile/20190424-183727/plugins/profile/2019-04-24_18-37-30/
logs/profile/20190424-183727/plugins/profile/2019-04-24_18-37-30/local.trace
logs/profile/20190424-183858/
logs/profile/20190424-183858/events.out.tfevents.1556131141.ea01b31b088f.profile-empty
logs/profile/20190424-183858/


Download **logs** directory again to see the new profile result in TensorBoard.

![TF Runtime](https://screenshot.googleplex.com/CYYS2Or914B.png)

The big **Iterator::GetNextSync** block is not there anymore.

Good job!

Apparently, this is still not the best performance yet. Please try by yourself to see if you can have further improvements.

## Other ways for profiling
In addition to TensorBoard callback, TensorFlow also provides two additional way to trigger profiler manually: **Profiler APIs** and **Profiler Service**.

**NOTE**: Please don't run multiple profilers at the same time. If you want to use either Profiler APIs or Profiler Service with TensorBoard callback, please create TensorBoard callback with **profile_batch=0**.

### Profiler APIs




In [0]:
# Context manager APIs
with tf.python.eager.profiler.Profiler('logdir_path'):
  # do your training here
  pass


# Function APIs
tf.python.eager.profiler.start()
# do your training here
profiler_result = tf.python.eager.profiler.stop()
tf.python.eager.profiler.save('logdir_path', profiler_result)

### Profiler Service



In [0]:
# This API will start a gRPC server with your TensorFlow job which can receive
# on-demand profiling request.
tf.python.eager.profiler.start_profiler_server(6009)

# Your TensorFlow program here

Then you can send profiling request to the port to perform on-demand profiling on TensorBoard by clicking **CAPTURE PROFILE** button:

![CAPTURE PROFILE](https://screenshot.googleplex.com/bWpOCoo41mp.png)

After successfully captured, refresh TensorBoard to visualize the result.